In [1]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, CSVLogger
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, LSTM, Dense
from data_utils import DataSet

import numpy as np
import os.path
import time

In [2]:
def load_cnn_model():
    base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(480, 640, 3), pooling='avg')
    
    print(base_model.summary())
    outputs = base_model.get_layer('global_average_pooling2d').output # 'avg_pool'

    cnn_model = Model(inputs=base_model.input, outputs=outputs)

    return cnn_model


In [3]:
cnn_model = load_cnn_model()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 480, 640, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 239, 319, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 239, 319, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 239, 319, 32) 0           batch_normalization[0][0]        
_______________________________________________________________________________________

# Creating Sequences

## Looks to be ~ 12 seconds per video

### ~24 minutes for 120 videos, ~6.6 hrs for entire 1980 videos

In [4]:
# ---- RUN THE FOLLOWING CELLS TO GENERATE .NPY SEQ FOR ALL VIDEOS IN DATASET ----- # 
#      SEQUENCES THAT ALREADY EXIST WILL BE SKIPPED

seq_length = 16
dataset = DataSet(cnn_model)

In [5]:
dataset.data

[['train', 'backhand', 'p41_backhand_s3'],
 ['train', 'backhand', 'p23_backhand_s3'],
 ['train', 'backhand', 'p15_backhand_s1'],
 ['train', 'backhand', 'p25_backhand_s1'],
 ['train', 'backhand', 'p49_backhand_s1'],
 ['train', 'backhand', 'p43_backhand_s3'],
 ['train', 'backhand', 'p53_backhand_s2'],
 ['train', 'backhand', 'p28_backhand_s2'],
 ['train', 'backhand', 'p12_backhand_s1'],
 ['train', 'backhand', 'p12_backhand_s2'],
 ['train', 'backhand', 'p18_backhand_s2'],
 ['train', 'backhand', 'p42_backhand_s3'],
 ['train', 'backhand', 'p26_backhand_s2'],
 ['train', 'backhand', 'p47_backhand_s1'],
 ['train', 'backhand', 'p38_backhand_s1'],
 ['train', 'backhand', 'p14_backhand_s2'],
 ['train', 'backhand', 'p13_backhand_s2'],
 ['train', 'backhand', 'p33_backhand_s1'],
 ['train', 'backhand', 'p31_backhand_s2'],
 ['train', 'backhand', 'p15_backhand_s3'],
 ['train', 'backhand', 'p53_backhand_s1'],
 ['train', 'backhand', 'p21_backhand_s1'],
 ['train', 'backhand', 'p48_backhand_s3'],
 ['train', 

In [6]:
tic = time.time()

for ind, sample in enumerate(dataset.data):
    
    path = os.path.join('data', 'sequences', sample[1], sample[2] + '-' + str(seq_length) + \
        '-features.npy')
    
    if os.path.isfile(path):
        print("Sequence: {} already exists".format(ind))
    else:
        print("Generating and saving sequence: {}".format(ind))
        sequence = dataset.extract_seq_features(sample)

print("Time Elapsed: {}".format(time.time() - tic))

# NOTE: The folders in `VIDEO_RGB` must be consolidated into the following folders in order for the data
# to retrieved properly according to the data_file.csv
# - backhand, bvolley, forehand, fvolley, service, smash
# OTHERWISE YOU WILL GET EMPTY SEQUENCES THAT CANNOT BE TRAINED ON

Sequence: 0 already exists
Sequence: 1 already exists
Sequence: 2 already exists
Sequence: 3 already exists
Sequence: 4 already exists
Sequence: 5 already exists
Generating and saving sequence: 6
data/sequences/backhand/p53_backhand_s2-16-features
16
(2048,)
Generating and saving sequence: 7
data/sequences/backhand/p28_backhand_s2-16-features
16
(2048,)
Generating and saving sequence: 8
data/sequences/backhand/p12_backhand_s1-16-features
16
(2048,)
Generating and saving sequence: 9
data/sequences/backhand/p12_backhand_s2-16-features
16
(2048,)
Generating and saving sequence: 10
data/sequences/backhand/p18_backhand_s2-16-features
16
(2048,)
Generating and saving sequence: 11
data/sequences/backhand/p42_backhand_s3-16-features
16
(2048,)
Generating and saving sequence: 12
data/sequences/backhand/p26_backhand_s2-16-features
16
(2048,)
Generating and saving sequence: 13
data/sequences/backhand/p47_backhand_s1-16-features
16
(2048,)
Generating and saving sequence: 14
data/sequences/backhand

data/sequences/backhand/p20_backhand_s3-16-features
16
(2048,)
Generating and saving sequence: 89
data/sequences/backhand/p26_backhand_s3-16-features
16
(2048,)
Generating and saving sequence: 90
data/sequences/backhand/p37_backhand_s3-16-features
16
(2048,)
Generating and saving sequence: 91
data/sequences/backhand/p43_backhand_s2-16-features
16
(2048,)
Generating and saving sequence: 92
data/sequences/backhand/p9_backhand_s2-16-features
16
(2048,)
Generating and saving sequence: 93
data/sequences/backhand/p6_backhand_s2-16-features
16
(2048,)
Generating and saving sequence: 94
data/sequences/backhand/p24_backhand_s2-16-features
16
(2048,)
Generating and saving sequence: 95
data/sequences/backhand/p40_backhand_s2-16-features
16
(2048,)
Generating and saving sequence: 96
data/sequences/backhand/p3_backhand_s3-16-features
16
(2048,)
Generating and saving sequence: 97
data/sequences/backhand/p33_backhand_s2-16-features
16
(2048,)
Generating and saving sequence: 98
data/sequences/backhand

data/sequences/backhand/p6_backhand2h_s2-16-features
16
(2048,)
Generating and saving sequence: 172
data/sequences/backhand/p5_backhand2h_s3-16-features
16
(2048,)
Generating and saving sequence: 173
data/sequences/backhand/p42_backhand2h_s2-16-features
16
(2048,)
Generating and saving sequence: 174
data/sequences/backhand/p8_backhand2h_s3-16-features
16
(2048,)
Generating and saving sequence: 175
data/sequences/backhand/p13_backhand2h_s3-16-features
16
(2048,)
Generating and saving sequence: 176
data/sequences/backhand/p19_backhand2h_s1-16-features
16
(2048,)
Generating and saving sequence: 177
data/sequences/backhand/p35_backhand2h_s3-16-features
16
(2048,)
Generating and saving sequence: 178
data/sequences/backhand/p45_backhand2h_s2-16-features
16
(2048,)
Generating and saving sequence: 179
data/sequences/backhand/p26_backhand2h_s3-16-features
16
(2048,)
Generating and saving sequence: 180
data/sequences/backhand/p23_backhand2h_s1-16-features
16
(2048,)
Generating and saving sequenc

data/sequences/backhand/p31_backhand2h_s2-16-features
16
(2048,)
Generating and saving sequence: 254
data/sequences/backhand/p33_backhand2h_s3-16-features
16
(2048,)
Generating and saving sequence: 255
data/sequences/backhand/p12_backhand2h_s1-16-features
16
(2048,)
Generating and saving sequence: 256
data/sequences/backhand/p11_backhand2h_s1-16-features
16
(2048,)
Generating and saving sequence: 257
data/sequences/backhand/p43_backhand2h_s1-16-features
16
(2048,)
Generating and saving sequence: 258
data/sequences/backhand/p26_backhand2h_s1-16-features
16
(2048,)
Generating and saving sequence: 259
data/sequences/backhand/p14_backhand2h_s1-16-features
16
(2048,)
Generating and saving sequence: 260
data/sequences/backhand/p46_backhand2h_s1-16-features
16
(2048,)
Generating and saving sequence: 261
data/sequences/backhand/p38_backhand2h_s3-16-features
16
(2048,)
Generating and saving sequence: 262
data/sequences/backhand/p28_backhand2h_s3-16-features
16
(2048,)
Generating and saving sequ

data/sequences/backhand/p10_bslice_s2-16-features
16
(2048,)
Generating and saving sequence: 336
data/sequences/backhand/p37_bslice_s3-16-features
16
(2048,)
Generating and saving sequence: 337
data/sequences/backhand/p32_bslice_s3-16-features
16
(2048,)
Generating and saving sequence: 338
data/sequences/backhand/p9_bslice_s3-16-features
16
(2048,)
Generating and saving sequence: 339
data/sequences/backhand/p9_bslice_s2-16-features
16
(2048,)
Generating and saving sequence: 340
data/sequences/backhand/p31_bslice_s3-16-features
16
(2048,)
Generating and saving sequence: 341
data/sequences/backhand/p45_bslice_s1-16-features
16
(2048,)
Generating and saving sequence: 342
data/sequences/backhand/p32_bslice_s1-16-features
16
(2048,)
Generating and saving sequence: 343
data/sequences/backhand/p3_bslice_s3-16-features
16
(2048,)
Generating and saving sequence: 344
data/sequences/backhand/p18_bslice_s3-16-features
16
(2048,)
Generating and saving sequence: 345
data/sequences/backhand/p40_bslic

...

data/sequences/service/p47_serkick_s3-16-features
16
(2048,)
Generating and saving sequence: 1597
data/sequences/service/p9_serkick_s1-16-features
16
(2048,)
Generating and saving sequence: 1598
data/sequences/service/p17_serkick_s2-16-features
16
(2048,)
Generating and saving sequence: 1599
data/sequences/service/p54_serkick_s2-16-features
16
(2048,)
Generating and saving sequence: 1600
data/sequences/service/p14_serkick_s3-16-features
16
(2048,)
Generating and saving sequence: 1601
data/sequences/service/p30_serkick_s3-16-features
16
(2048,)
Generating and saving sequence: 1602
data/sequences/service/p15_serkick_s1-16-features
16
(2048,)
Generating and saving sequence: 1603
data/sequences/service/p14_serkick_s2-16-features
16
(2048,)
Generating and saving sequence: 1604
data/sequences/service/p17_serkick_s3-16-features
16
(2048,)
Generating and saving sequence: 1605
data/sequences/service/p46_serkick_s1-16-features
16
(2048,)
Generating and saving sequence: 1606
data/sequences/servic

data/sequences/service/p34_serslice_s2-16-features
16
(2048,)
Generating and saving sequence: 1681
data/sequences/service/p9_serslice_s3-16-features
16
(2048,)
Generating and saving sequence: 1682
data/sequences/service/p47_serslice_s3-16-features
16
(2048,)
Generating and saving sequence: 1683
data/sequences/service/p16_serslice_s2-16-features
16
(2048,)
Generating and saving sequence: 1684
data/sequences/service/p25_serslice_s2-16-features
16
(2048,)
Generating and saving sequence: 1685
data/sequences/service/p55_serslice_s3-16-features
16
(2048,)
Generating and saving sequence: 1686
data/sequences/service/p43_serslice_s3-16-features
16
(2048,)
Generating and saving sequence: 1687
data/sequences/service/p30_serslice_s3-16-features
16
(2048,)
Generating and saving sequence: 1688
data/sequences/service/p39_serslice_s2-16-features
16
(2048,)
Generating and saving sequence: 1689
data/sequences/service/p14_serslice_s3-16-features
16
(2048,)
Generating and saving sequence: 1690
data/sequen

data/sequences/service/p29_serslice_s2-16-features
16
(2048,)
Generating and saving sequence: 1764
data/sequences/service/p52_serslice_s1-16-features
16
(2048,)
Generating and saving sequence: 1765
data/sequences/service/p46_serslice_s2-16-features
16
(2048,)
Generating and saving sequence: 1766
data/sequences/service/p50_serslice_s1-16-features
16
(2048,)
Generating and saving sequence: 1767
data/sequences/service/p53_serslice_s3-16-features
16
(2048,)
Generating and saving sequence: 1768
data/sequences/service/p12_serslice_s2-16-features
16
(2048,)
Generating and saving sequence: 1769
data/sequences/service/p6_serslice_s2-16-features
16
(2048,)
Generating and saving sequence: 1770
data/sequences/service/p12_serslice_s3-16-features
16
(2048,)
Generating and saving sequence: 1771
data/sequences/service/p46_serslice_s3-16-features
16
(2048,)
Generating and saving sequence: 1772
data/sequences/service/p44_serslice_s3-16-features
16
(2048,)
Generating and saving sequence: 1773
data/sequen

data/sequences/smash/p54_smash_s3-16-features
16
(2048,)
Generating and saving sequence: 1849
data/sequences/smash/p19_smash_s2-16-features
16
(2048,)
Generating and saving sequence: 1850
data/sequences/smash/p37_smash_s1-16-features
16
(2048,)
Generating and saving sequence: 1851
data/sequences/smash/p42_smash_s1-16-features
16
(2048,)
Generating and saving sequence: 1852
data/sequences/smash/p2_smash_s1-16-features
16
(2048,)
Generating and saving sequence: 1853
data/sequences/smash/p37_smash_s3-16-features
16
(2048,)
Generating and saving sequence: 1854
data/sequences/smash/p19_smash_s3-16-features
16
(2048,)
Generating and saving sequence: 1855
data/sequences/smash/p22_smash_s3-16-features
16
(2048,)
Generating and saving sequence: 1856
data/sequences/smash/p50_smash_s3-16-features
16
(2048,)
Generating and saving sequence: 1857
data/sequences/smash/p10_smash_s1-16-features
16
(2048,)
Generating and saving sequence: 1858
data/sequences/smash/p17_smash_s1-16-features
16
(2048,)
Gene

data/sequences/smash/p23_smash_s3-16-features
16
(2048,)
Generating and saving sequence: 1937
data/sequences/smash/p53_smash_s2-16-features
16
(2048,)
Generating and saving sequence: 1938
data/sequences/smash/p18_smash_s2-16-features
16
(2048,)
Generating and saving sequence: 1939
data/sequences/smash/p15_smash_s2-16-features
16
(2048,)
Generating and saving sequence: 1940
data/sequences/smash/p3_smash_s1-16-features
16
(2048,)
Generating and saving sequence: 1941
data/sequences/smash/p18_smash_s3-16-features
16
(2048,)
Generating and saving sequence: 1942
data/sequences/smash/p13_smash_s1-16-features
16
(2048,)
Generating and saving sequence: 1943
data/sequences/smash/p27_smash_s2-16-features
16
(2048,)
Generating and saving sequence: 1944
data/sequences/smash/p24_smash_s2-16-features
16
(2048,)
Generating and saving sequence: 1945
data/sequences/smash/p30_smash_s2-16-features
16
(2048,)
Generating and saving sequence: 1946
data/sequences/smash/p29_smash_s3-16-features
16
(2048,)
Gene

# Sanity Check on Generated Sequences

In [7]:
path = os.path.join('data', 'sequences', 'backhand', 'p36_backhand_s3-16-features.npy')

sequence = np.load(path)
sequence

array([[0.6328085 , 0.33544427, 0.3047984 , ..., 0.27660713, 0.40139115,
        0.3712773 ],
       [0.6667429 , 0.2969837 , 0.24121533, ..., 0.29314616, 0.53533936,
        0.34506533],
       [0.70823026, 0.35714155, 0.31769595, ..., 0.30259904, 0.37390515,
        0.29302108],
       ...,
       [0.6553284 , 0.34075412, 0.2506684 , ..., 0.36052352, 0.5520055 ,
        0.4221629 ],
       [0.67981297, 0.36810282, 0.28184068, ..., 0.30228648, 0.5648418 ,
        0.3953815 ],
       [0.7253655 , 0.36406708, 0.24089433, ..., 0.31713542, 0.5203288 ,
        0.38690814]], dtype=float32)